In [66]:
import pandas, keras
import numpy as np
from sklearn.metrics import roc_auc_score
from IPython.display import FileLink, FileLinks

In [67]:
# 1 image has 16x16 pixels = 256 pixels
pixels = ["pixel_{0}".format(i) for i in range(36)]

def to_image(df):
    return  np.expand_dims(np.expand_dims(df[pixels], axis=-1).reshape(-1,6,6), axis=-1)


#store_train = pandas.HDFStore("SNG_p10_pixelTrain.h5")
store_train = pandas.HDFStore("pixelTrain.h5")
#store_train = pandas.HDFStore("pixelTrain.h5")

df_train = store_train.select("df",stop=-1)
df_train = df_train[(df_train["GenDeltaR"]<0.1) & (df_train["nUniqueSimTracksInSharedHit"]>-1)]
images_train = to_image(df_train)

store_test = pandas.HDFStore("pixelTrain30k.h5")

df_test = store_test.select("df",stop=-1)
df_test = df_test[(df_test["GenDeltaR"]<0.1) & (df_test["nUniqueSimTracksInSharedHit"]>-1)]#print(df_test)
images_test = to_image(df_test)

#comb = pandas.concat([df_train,df_test])

#df_train=comb.sample(frac=0.6)
#df_test=comb.drop(df_train.index)

#df_train = df_train[(df_train["GenDeltaR"]<0.1) & (df_train["nUniqueSimTracksInSharedHit"]>-1)]

# Make all test, train data merged hit
df_test = df_test[(df_test["isSharedHit"]>0)]
df_train = df_train[(df_train["isSharedHit"]>0)]

#images_train = to_image(df_train)
#images_test = to_image(df_test)


#print(df_train.iloc[[28946]])
#print(df_train)
#print df_train
#print sum(df_train["isSharedHit"])


In [68]:
store_train = pandas.HDFStore("pixelTrain.h5")

df_train = store_train.select("df",stop=-1)
df_train = df_train[(df_train["GenDeltaR"]<0.1) & (df_train["nUniqueSimTracksInSharedHit"]>-1)]

store_test = pandas.HDFStore("pixelTrain30k.h5")

df_test = store_test.select("df",stop=-1)
df_test = df_test[(df_test["GenDeltaR"]<0.1) & (df_test["nUniqueSimTracksInSharedHit"]>-1)]#print(df_test)

print df_test.shape
print df_train.shape

# Remove overlap between train and test data
df_combined = pandas.concat([df_test, df_train]).drop_duplicates(keep=False)
df_combined.shape


# New combined files
df_train=df_combined.sample(frac=0.6)
df_test=df_combined.drop(df_train.index)

# Make all test, train data merged hit
df_test = df_test[(df_test["isSharedHit"]>0)]
df_train = df_train[(df_train["isSharedHit"]>0)]

df_train = df_train[(df_train["GenDeltaR"]<0.1) & (df_train["nUniqueSimTracksInSharedHit"]>-1)]


images_train = to_image(df_train)
images_test = to_image(df_test)


(82, 42)
(313, 42)


In [69]:
df_test.shape

(113, 42)

In [70]:
df_augment_train = df_train.iloc[0:1]
augment_train = to_image(df_augment_train)


In [71]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
datagen = ImageDataGenerator(horizontal_flip=True)
datagen.fit(images_train, augment=True)

In [72]:
# Adding isMergedHit column 

merged_hit = df_train["nUniqueSimTracksInSharedHit"]>1
merged_int = merged_hit.astype(int)
df_train.insert(0, "isMergedHit", merged_int, True)

#df_train.head()

merged_hit_test = df_test["nUniqueSimTracksInSharedHit"]>1
merged_int_test = merged_hit_test.astype(int)
df_test.insert(0, "isMergedHit", merged_int_test, True)

df_test.head()

,isMergedHit,isSharedHit,trackPt,trackEta,trackPhi,nUniqueSimTracksInSharedHit,GenDeltaR,pixel_0,pixel_1,pixel_2,...,pixel_26,pixel_27,pixel_28,pixel_29,pixel_30,pixel_31,pixel_32,pixel_33,pixel_34,pixel_35
69,1,1,1.330709,-2.043563,1.330709,2.0,0.043240,0.0,0.0,0.087983,...,0.083277,0.139740,0.000000,0.0,0.0,0.0,0.04586,0.090908,0.0,0.0
92,0,1,1.927646,-2.203123,1.927646,1.0,0.016287,0.0,0.0,0.000000,...,0.000000,0.403630,0.056244,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0
119,1,1,4.657765,-1.873906,4.657765,2.0,0.005257,0.0,0.0,0.000000,...,0.254677,0.101919,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0
228,0,1,13.573482,-0.220184,13.573482,1.0,0.003009,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0
229,0,1,1.924295,-2.205955,1.924295,1.0,0.020649,0.0,0.0,0.000000,...,0.072539,0.114365,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0


In [73]:
# Check that all are merged hits i.e. isSharedHit=1
print df_test['isSharedHit'].head()
print df_train['isSharedHit'].head()

69     1
92     1
119    1
228    1
229    1
Name: isSharedHit, dtype: int32
1668    1
7480    1
5655    1
7993    1
433     1
Name: isSharedHit, dtype: int32


In [74]:
print df_test.shape
print df_train.shape

(113, 43)
(174, 43)


In [75]:
from keras.layers import Dense, Dropout, Activation
# Define the network
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size=(4,4), padding='same', activation='relu'))
#model.add(keras.layers.Conv2D(16, kernel_size=(4,4), padding='same', activation='relu'))
#model.add(keras.layers.Conv2D(16, kernel_size=(2,2), padding='same', activation='relu'))
#model.add(keras.layers.Conv2D(16, kernel_size=(1,1), padding='same', activation='relu'))
# max pooling 2D
#model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same', data_format="channels_last"))
model.add(keras.layers.Flatten(input_shape=(6,6,1),data_format = "channels_last"))
#model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
# dropout 10%
model.add(Dropout(0.1))
model.add(keras.layers.Dense(2, activation='softmax'))
#print(model.summary())

# Train the network
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics = ["accuracy"])
model.summary
model.fit(datagen.flow(images_train, keras.utils.to_categorical(df_train["nUniqueSimTracksInSharedHit"]>1)), epochs=30) # validation fraction
print(model.summary)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=<keras_preprocessing.image.numpy_array_iterator.NumpyArrayIterator object at 0x7fed1d258b90>

In [ ]:
#store_test = pandas.HDFStore("pixelTrain30k.h5")

#df_test = store_test.select("df",stop=-1)
##pixels
#df_test[pixels]

In [ ]:
### Evaluate performance on independent sample
# DO NOT CHANGE BELOW!

# Prepare input
#store_test = pandas.HDFStore("SNG_p10_pixelTrain.h5")


#store_test = pandas.HDFStore("pixelTest_v3.h5")
#store_test = pandas.HDFStore("pixelTrain30k.h5")

#df_test = store_test.select("df",stop=-1)
#df_test = df_test[(df_test["GenDeltaR"]<0.1) & (df_test["nUniqueSimTracksInSharedHit"]>-1)]#print(df_test)
#images_test = to_image(df_test)
#images_test = pandas.read_hdf("pixelTrain.h5")
#images_test = images_train
#df_test = df_train

# Run DNN
print("Running on full test sample. This may take a moment.")
ret = model.predict(images_test)
np.save("result.npy",ret[:,1])
print(ret)
#print ret
#print df_test["isSharedHit"]
#keras.utils.to_categorical(df_test["isSharedHit"])
#print len(keras.utils.to_categorical(df_test["isSharedHit"]))
#print keras.utils.to_categorical(df_train["isSharedHit"],dtype='int32').sum()
#print len(keras.utils.to_categorical(df_train["isSharedHit"])),keras.utils.to_categorical(df_train["isSharedHit"]).sum()

In [ ]:
from sklearn.metrics import roc_curve
#fpr_keras, tpr_keras, thresholds_keras = roc_curve(keras.utils.to_categorical(df_test["isSharedHit"])[:,1], ret[:,1])
fpr_keras, tpr_keras, thresholds_keras = roc_curve(keras.utils.to_categorical(df_test["nUniqueSimTracksInSharedHit"]>1)[:,1], ret[:,1])

print fpr_keras,tpr_keras
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)
print auc_keras
print np.isnan(fpr_keras).all()
print len(fpr_keras),len(tpr_keras)
np.save("fpr_keras.npy",fpr_keras)
np.save("tpr_keras.npy",tpr_keras)


In [ ]:
import numpy as np
fpr_keras = np.load("fpr_keras.npy")
tpr_keras = np.load("tpr_keras.npy")
#print fpr_keras[2300]
auc = np.trapz(tpr_keras,fpr_keras)
print auc
#auc_keras = 0.623416233325797
#auc_keras = 0.665570454761781
#print test
import matplotlib.pyplot as plt
plt.figure(1)
#plt.figure(figsize=(200, 200))
#plt.plot([0, 1], [0, 1], label='test')
#plt.plot([1,2,3],[5,6,7], label = 'test')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc))
#plt.plot(fpr_keras, tpr_keras, label='test')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (area = {:.3f})'.format(auc))
#plt.legend(loc='best')
plt.savefig("ROC.png")
plt.savefig("ROC.pdf")

plt.show()


In [ ]:
#sanity check: no overlap between train and test
pandas.merge(df_train, df_test, on=[x for x in df_train.columns], how='inner')

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
#df_test[(df_test["nUniqueSimTracksInSharedHit"]>-1) & (df_test["GenDeltaR"]<0.1)]

In [ ]:
print "shared hits with at least 2 sim tracks:"
print float(sum(df_train["nUniqueSimTracksInSharedHit"]>1))/len(df_train["nUniqueSimTracksInSharedHit"])
plt.hist(df_train["nUniqueSimTracksInSharedHit"],histtype="step",bins=6,range=(-0.5,5.5))
plt.title("nUniqueSimTracksInSharedHit")

In [ ]:
sum(df_train["nUniqueSimTracksInSharedHit"]>-1)

In [ ]:
82.0/30000

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print float(sum(df_train["nUniqueSimTracksInSharedHit"]>1))/len(df_train["nUniqueSimTracksInSharedHit"])
plt.hist(df_train["nUniqueSimTracksInSharedHit"]>1,histtype="step",bins=2,range=(-0.5,1.5))
plt.title("Distribution of Hits in Training Data")
plt.xticks([0,1],("Not Merged","Merged"))
plt.savefig("merged_dist.png")
plt.savefig("merged_dist.pdf")

In [ ]:
# Separating signal and background for train and test data
signal = ret[df_test["isMergedHit"]==1]
background = ret[df_test["isMergedHit"]==0]

signal_plt = signal[:,1]
background_plt = background[:,1]

ret_train = model.predict(images_train)

signal_train = ret_train[df_train["isMergedHit"]==1]
background_train = ret_train[df_train["isMergedHit"]==0]
signal_train_plt = signal_train[:,1]
background_train_plt = background_train[:,1]

Y_back_hist = np.histogram(background_train_plt)[0]
X_back_hist = np.histogram(background_train_plt)[1]

Y_sig_hist = np.histogram(signal_train_plt)[0]
X_sig_hist = np.histogram(signal_train_plt)[1]

In [ ]:
#Signal and background plot
plt.hist(signal_plt, alpha = 0.5, color = 'b', label = 'Signal (test)', range = (0,1), align = 'left')
plt.hist(background_plt, color = 'r', alpha = 0.5, label = 'Background (test)', range = (0,1))
plt.scatter(X_back_hist[0:10], Y_back_hist, label='Background (train)', color ='r')
plt.scatter(X_sig_hist[0:10], Y_sig_hist, label='Signal (train)', color='b')
plt.legend(loc='best')
plt.title('CNN Signal and Background')

In [ ]:
plt.hist(signal_train_plt, align = 'right')
plt.scatter([0.1782944 , 0.26801087, 0.35772735, 0.44744383,
        0.5371603 , 0.62687678, 0.71659326, 0.80630973, 0.89602621,
        0.98574269], [ 3.,  2.,  1.,  2.,  2.,  1.,  8., 14., 17., 42.], )

In [ ]:
print signal_plt

In [ ]:
print df_test.shape
print df_train.shape

In [ ]:
ret